In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

def getData():
    
    urls = []
    url = 'https://www.pro-football-reference.com/schools/'
    soup = bs(requests.get(url).text, 'html.parser')
    tags = soup.find_all('td', {'data-stat':'college_name'})
    for tag in tags:
        if tag.contents[0] != 'none':
            urls.append('https://www.pro-football-reference.com'+tag.contents[0]['href'])

    teams = []
    players = []
    positions = []
    years = []
    pbs = []
    starts = []
    cavgs = []

    for u in urls[:200]:

        soup = bs(requests.get(u).text, 'html.parser')
        tags = soup.find_all('th', {'data-stat':'pos'})

        for t in tags:
            pos = t.text.strip()
            if pos[:2] in ['FS', 'SS', 'CB', 'DB']:
                tag = t.parent
                teams.append(soup.find_all('h1', {'itemprop':'name'})[0].text.strip().split('\n')[0])
                players.append(tag.find_all('td',{'data-stat':'player'})[0].find_all('a')[0].text.strip())
                positions.append(pos)
                years.append(int(tag.find_all('td',{'data-stat':'years_played'})[0].text.strip()[:4]))
                pbs.append(int(tag.find_all('td',{'data-stat':'pro_bowls'})[0].text.strip()))
                starts.append(int(tag.find_all('td',{'data-stat':'years_as_primary_starter'})[0].text.strip()))
                cavgs.append(int(tag.find_all('td',{'data-stat':'career_av'})[0].text.strip()))


    return pd.DataFrame({'Team':teams, 'Player':players, 'Position':positions, 'Year':years, \
                         'Pro Bowls':pbs, 'Starts':starts, 'Career Value':cavgs})

In [2]:
df = getData()
df

,Team,Player,Position,Year,Pro Bowls,Starts,Career Value
0,Notre Dame,Pat Ballage,DB,1986,0,0,1
1,Notre Dame,Robert Blanton,CB,2012,0,1,9
2,Notre Dame,Luther Bradley,DB,1978,0,2,14
3,Notre Dame,Chris Brown,DB,1984,0,0,3
4,Notre Dame,Sergio Brown,DB,2010,0,1,10
5,Notre Dame,Jim Browner,DB,1979,0,0,3
6,Notre Dame,David Bruton,DB,2009,0,0,10
7,Notre Dame,Frank Budka,DB,1964,0,0,2
8,Notre Dame,Ted Burgmeier,DB,1978,0,0,1
9,Notre Dame,Jeff Burris,DB,1994,0,9,41


In [3]:
teams = list(df.groupby('Team')['Team'].count().index)
counts = list(df.groupby('Team')['Team'].count())
avgVals = list(df.groupby('Team')['Career Value'].mean())

redux = pd.DataFrame({'Team':teams, '# of Players':counts, 'Avg Career Value': avgVals})
redux

,Team,# of Players,Avg Career Value
0,Abilene Christian,3,15.333333
1,Akron,6,15.666667
2,Alabama,54,14.277778
3,Alabama St.,3,5.000000
4,Appalachian St.,8,5.125000
5,Arizona,27,14.629630
6,Arizona St.,48,18.750000
7,Ark-Pine Bluff,5,13.800000
8,Arkansas,26,13.576923
9,Arkansas St.,11,4.909091


In [4]:
playerMax = redux.sort_values(by = '# of Players').iloc[-1,:]
print(playerMax['Team'], 'had the most players in the NFL at', playerMax['# of Players'])
valMax = redux.sort_values(by = 'Avg Career Value').iloc[-1,:]
print(valMax['Team'], 'had the highest avg valued players in the NFL at', valMax['Avg Career Value'])


Ohio St. had the most players in the NFL at 72
Colgate had the highest avg valued players in the NFL at 65.5


In [5]:
import statistics as stat
outlier = 2*stat.stdev(counts)+stat.mean(counts)
redux[(redux['# of Players'] > outlier)].sort_values(by = 'Avg Career Value', ascending = False)

,Team,# of Players,Avg Career Value
85,Michigan,55,18.290909
165,USC,68,17.617647
152,Texas,52,17.538462
114,Ohio St.,72,16.930556
69,LSU,71,15.943662
83,Miami (FL),59,15.813559
163,UCLA,57,15.701754
2,Alabama,54,14.277778
43,Florida,55,12.836364
112,Notre Dame,63,12.396825
